In [6]:
%reload_kedro
from twitter_network.nodes.features import *
from sklearn.feature_selection import GenericUnivariateSelect, SelectFromModel, mutual_info_classif

2019-07-03 11:33:15,723 - INFO - ** Kedro project Twitter Network
2019-07-03 11:33:15,725 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/logging.yml
2019-07-03 11:33:15,730 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/catalog.yml
2019-07-03 11:33:15,735 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/credentials.yml
2019-07-03 11:33:15,737 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/parameters.yml
2019-07-03 11:33:15,743 - INFO - Defined global variables proj_dir, proj_name, conf and io


In [3]:
X_train = io.load("stand_log_X_train")
X_valid = io.load("stand_log_X_valid")
X_test = io.load("stand_log_X_test")
y_train = io.load("y_train")
print('Number of features:', X_train.shape[1])

2019-07-03 11:32:46,805 - INFO - Loading data from `stand_log_X_train` (PickleLocalDataSet)...
2019-07-03 11:32:46,817 - INFO - Loading data from `stand_log_X_valid` (PickleLocalDataSet)...
2019-07-03 11:32:46,825 - INFO - Loading data from `stand_log_X_test` (PickleLocalDataSet)...
2019-07-03 11:32:46,830 - INFO - Loading data from `y_train` (PickleLocalDataSet)...
Number of features: 68


In [11]:
MI = pd.DataFrame(mutual_info_classif(X_train, y_train), index = X_train.columns, columns = ["MI"])
MI_order = list(MI.sort_values("MI", ascending = False).index)

['sink_page_rank',
 'sink_katz',
 'sink_centrality',
 'log_sink_neighbors',
 'sink_in_centrality',
 'log_sink_in_degree',
 'RA_undirected',
 'RA_neighbors',
 'AA_neighbors',
 'AA_undirected',
 'RA_predecessors',
 'RA_transient_in',
 'AA_predecessors',
 'link_efficiency',
 'AA_transient_in',
 'log_common_neighbors',
 'hub_promoted_index',
 'hub_suppressed_index',
 'sorensen_index',
 'JC_neighbors',
 'JC_undirected',
 'PA_predecessors',
 'PA_neighbors',
 'PA_undirected',
 'shortest_path',
 'JC_transient_in',
 'transitive_links',
 'JC_predecessors',
 'cos_neighbors',
 'cos_predecessors',
 'LHN_index',
 'cos_transient_in',
 'PA_transient_in',
 'sink_in_density',
 'sink_bi_density',
 'sink_out_density',
 'log_sink_bi_degree',
 'log_sink_out_degree',
 'log_sink_avg_nbr_degree',
 'sink_reciprocity',
 'PA_successors',
 'PA_transient_out',
 'JC_successors',
 'sink_out_centrality',
 'AA_successors',
 'cos_successors',
 'RA_successors',
 'AA_transient_out',
 'JC_transient_out',
 'cos_transient_ou

In [26]:
substrings = ["AA","JC","RA","PA","cos","source_in","source_out","source_bi","sink_in","sink_out","sink_bi"]
sim_order = [[x for x in MI_order if substring in x] for substring in substrings]
sim_best = [x[0] for x in sim_order]
sim_exclude = [x[1:] for x in sim_order]
sim_exclude = [x for sublist in sim_exclude for x in sublist]
print('Best:', sim_best)
print('Excluded:', sim_exclude)

Best: ['AA_neighbors', 'JC_neighbors', 'RA_undirected', 'PA_predecessors', 'cos_neighbors', 'log_source_in_degree', 'log_source_out_degree', 'source_bi_density', 'sink_in_centrality', 'sink_out_density', 'sink_bi_density']
Excluded: ['AA_undirected', 'AA_predecessors', 'AA_transient_in', 'AA_successors', 'AA_transient_out', 'JC_undirected', 'JC_transient_in', 'JC_predecessors', 'JC_successors', 'JC_transient_out', 'RA_neighbors', 'RA_predecessors', 'RA_transient_in', 'RA_successors', 'RA_transient_out', 'PA_neighbors', 'PA_undirected', 'PA_transient_in', 'PA_successors', 'PA_transient_out', 'cos_predecessors', 'cos_transient_in', 'cos_successors', 'cos_transient_out', 'source_in_centrality', 'source_in_density', 'source_out_centrality', 'source_out_density', 'log_source_bi_degree', 'log_sink_in_degree', 'sink_in_density', 'log_sink_out_degree', 'sink_out_centrality', 'log_sink_bi_degree']


In [27]:
features = X_train.columns
included = [x for x in features if x not in sim_exclude]; included

['AA_neighbors',
 'JC_neighbors',
 'LHN_index',
 'PA_predecessors',
 'RA_undirected',
 'cos_neighbors',
 'hub_promoted_index',
 'hub_suppressed_index',
 'is_followed_back',
 'link_efficiency',
 'log_common_neighbors',
 'log_node_boundary_size',
 'log_sink_avg_nbr_degree',
 'log_sink_neighbors',
 'log_source_avg_nbr_degree',
 'log_source_in_degree',
 'log_source_neighbors',
 'log_source_out_degree',
 'log_total_neighbors',
 'shortest_path',
 'sink_bi_density',
 'sink_centrality',
 'sink_in_centrality',
 'sink_katz',
 'sink_out_density',
 'sink_page_rank',
 'sink_reciprocity',
 'sorensen_index',
 'source_bi_density',
 'source_centrality',
 'source_katz',
 'source_page_rank',
 'source_reciprocity',
 'transitive_links']

In [ ]:
# Get feature mask

mask = selector.get_support()
selected = list(compress(features, mask))
excluded = list(set(features) - set(selected))
print('Selected variables:', selected, '\n')
print('Excluded variables:', excluded, '\n')

In [32]:
%reload_kedro
clf = io.load("clf_RF")
clf.feature_importances_

2019-07-03 11:43:26,983 - INFO - ** Kedro project Twitter Network
2019-07-03 11:43:26,985 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/logging.yml
2019-07-03 11:43:26,990 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/catalog.yml
2019-07-03 11:43:26,995 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/credentials.yml
2019-07-03 11:43:26,997 - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/Machine Learning/twitter-network/conf/base/parameters.yml
2019-07-03 11:43:27,003 - INFO - Defined global variables proj_dir, proj_name, conf and io
2019-07-03 11:43:27,004 - INFO - Loading data from `clf_RF` (PickleLocalDataSet)...


array([3.84151304e-02, 1.91672173e-02, 5.27549094e-05, 1.94297675e-02,
       3.44471696e-05, 5.72616620e-02, 9.78268235e-03, 3.08249480e-03,
       4.08474812e-05, 1.85148921e-03, 2.37708649e-05, 1.18790452e-02,
       5.16871012e-03, 7.91600129e-03, 1.73647510e-02, 2.55198427e-04,
       9.38548351e-03, 1.13438082e-03, 7.16841295e-03, 4.95477478e-02,
       2.80422697e-02, 4.77993390e-05, 3.38805601e-02, 1.80849418e-05,
       7.26355001e-02, 4.10469278e-03, 3.99411236e-03, 4.39915037e-05,
       1.43317399e-03, 2.55039481e-05, 7.26390854e-03, 1.76468832e-02,
       1.48163658e-04, 1.20352909e-02, 2.03961342e-02, 7.11828167e-03,
       4.86494764e-04, 9.21510967e-04, 2.25642752e-02, 2.43993738e-02,
       6.07323133e-04, 6.73693654e-03, 6.60620152e-03, 6.55728711e-03,
       7.06892660e-03, 7.31801003e-03, 6.57438277e-03, 6.14784004e-03,
       7.42685030e-04, 2.87190183e-02, 2.29010664e-02, 1.84947471e-02,
       1.25744720e-01, 4.20224459e-04, 5.69113029e-04, 1.63847192e-01,
      

In [33]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)
X_reduced = model.transform(X_train)
X_reduced.shape

(39240, 29)

In [34]:
mask = model.get_support()
selected = list(compress(features, mask))
excluded = list(set(features) - set(selected))
print('Selected variables:', selected, '\n')
print('Excluded variables:', excluded, '\n')

Selected variables: ['JC_transient_in', 'JC_transient_out', 'LHN_index', 'PA_transient_in', 'RA_transient_in', 'cos_neighbors', 'hub_promoted_index', 'is_followed_back', 'link_efficiency', 'log_common_neighbors', 'log_sink_bi_degree', 'log_sink_in_degree', 'log_sink_out_degree', 'log_source_avg_nbr_degree', 'log_source_bi_degree', 'log_source_in_degree', 'log_source_out_degree', 'shortest_path', 'sink_bi_density', 'sink_in_centrality', 'sink_in_density', 'sink_out_density', 'sink_page_rank', 'sink_reciprocity', 'source_in_density', 'source_katz', 'source_out_centrality', 'source_out_density', 'transitive_links'] 

Excluded variables: ['sink_centrality', 'source_bi_density', 'AA_transient_in', 'sink_out_centrality', 'AA_successors', 'RA_undirected', 'cos_predecessors', 'source_page_rank', 'sorensen_index', 'cos_transient_in', 'PA_predecessors', 'PA_neighbors', 'AA_neighbors', 'source_reciprocity', 'log_sink_neighbors', 'RA_successors', 'AA_transient_out', 'AA_undirected', 'RA_neighbors'

In [46]:
from sklearn.feature_selection import RFE
clf = io.load("clf_GB")
print("Fitting RFE...")
rfe = RFE(estimator=clf, n_features_to_select=30, step=1)
rfe.fit(X_train, y_train)
print("Ranking:", rfe.ranking_)

2019-07-03 13:45:08,227 - INFO - Loading data from `clf_GB` (PickleLocalDataSet)...
Fitting RFE...


KeyboardInterrupt: 

In [43]:
clf = io.load("clf_GB")
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

2019-07-03 13:44:08,546 - INFO - Loading data from `clf_GB` (PickleLocalDataSet)...


In [48]:
list(features[indices])[:30]

['sink_page_rank',
 'sink_in_density',
 'RA_undirected',
 'RA_neighbors',
 'source_out_centrality',
 'log_node_boundary_size',
 'RA_transient_in',
 'sink_katz',
 'sink_in_centrality',
 'log_source_bi_degree',
 'log_source_out_degree',
 'log_sink_in_degree',
 'cos_neighbors',
 'sink_centrality',
 'PA_predecessors',
 'log_source_in_degree',
 'source_in_density',
 'AA_transient_in',
 'RA_predecessors',
 'JC_predecessors',
 'link_efficiency',
 'cos_predecessors',
 'PA_transient_in',
 'JC_neighbors',
 'shortest_path',
 'source_bi_density',
 'log_sink_neighbors',
 'source_in_centrality',
 'JC_transient_in',
 'PA_undirected']